## Tutorial on sleep spindle analysis using YASA
(c) Simon Kern 2022

Welcome to this short tutorial on getting started with Jupyter Notebook and YASA. This tutorial will try to provide a beginner-friendly instruction on how to use Jupyter notebooks to run sleep spindle analysis. It is be recommended to watch a short introduction on Jupyter notebooks before running this notebook to understand the basics. For German speakers this one seems well suited: https://www.youtube.com/watch?v=1S4Cgtkxqhs 

Generally, Jupyter notebooks allow to subdivide Python code into different cells that can be run individually and directly show the output of the given code. Additionally, cells with human-readable explanations (such as this cell itself) are possible. You can execute a cell by pressing SHIFT-ENTER. If it's a code cell, the code inside will be executed, if it's a human readable cell (i.e. a MarkDown cell) it will render the code. 
1. Try double-clicking this text to enter the edit mode. 
2. As you can see, you now can edit the text and formatting 
3. Press STRG+ENTER or SHIFT-ENTER to render the text again

### Getting started

Before running the analysis we need to tell Python which packages it needs import. Select the cell below and press "STRG+ENTER" to execute it. You will see a star appearing left to it `In [*]` which will indicate the current cell is running. After a successful run it will display `In [1]` or any other number

In [ ]:
import mne # MNE is a package for processing EEG data in Python (e.g. loading EDF files)
import yasa # the package we previously installed, which contains most of the code we needy
import sleep_utils # this is a local package which is part of the GitHub repository
print('importing done!')

#### Loading a sleep recording
Now as an example we load a single EDF file and run a sleep spindle analysis on it using YASA.

For this you first need to first adapt the `data_dir` where the EDFs can be found and then execute the cell.

In [1]:
data_dir = 'Z:/Exercise_Sleep_Project/EDF Export EEG/' # adapt the path on the left to where the data can be found on your computer

edf_file = data_dir + '/AA3_EX_AA3_EX_(1).edf'  # this is simply the first EDF file in the list

print('Loading data, this might take a while...')
raw = mne.io.read_raw_edf(edf_file, preload=True) # we load the file into memory using the function `mne.io.read_raw_edf`
print('Done Loading')

Loading data, this might take a while...


NameError: name 'mne' is not defined

Now using the `print` command, we can display some information about this file


In [ ]:
# the object with the name `raw` is now a reference to the EEG containing all data, channels and other information
print(raw.info) # in the info object for example we will find e.g the sampling frequency and channel list

### Using YASA for spindle detection

Now we can use YASA to run some basic spindle analysis on one of the channels. We do this by simply calling this function:

```Python
yasa.spindles_detect(data_of_one_channel, sf=sampling_frequency)
```

To do this, we
1. Extract the data of a channel of interest from the RAW object, rescaled to uV (microvolt)
2. Apply a bipolar reference to this channel (assuming no reference had been applied previously)
3. Substract the reference channel from the channel data
4. Feed in this data into the spindle detect algorithm


Additionally, there are many more parameters that the `spindle_detect` function takes. However it has some sensible defaults that we just use for now. For a list of parameters you can look here: [documentation:yasa.spindles_detect](https://raphaelvallat.com/yasa/build/html/generated/yasa.spindles_detect.html)

These are the parameters that it uses by default
```Python
freq_sp      = (12,15)  # look for spindles between 12-15 Hz
freq_broad   = (1,30)   # filter signal between 1 to 30 Hz
duration     = (0.5, 2) # spindles should be between 0.5-2 seconds long
min_distance = 0.5      # minimum distance between two spindles is 0.5 seconds

```




In [ ]:
ch = 'C4' # let's look at C4
ref = 'M1' # take M1 as reference

data_ch  = raw.get_data(ch, units='uV')  # retrieve data from the channel
data_ref = raw.get_data(ref, units='uV') # retrieve data from the reference channel

data = data_ch-data_ref # subtract the two signals (bipolar EEG reference)

sampling_frequency = raw.info['sfreq'] # get the sampling frequency for these channels

# run the spindle detection with the above mentioned defaults
spindles = yasa.spindles_detect(data, sf=sampling_frequency) 

The `spindles` variable now contains the results of the simple spindle detection, let's have a look at it

In [ ]:
spindles_summary = spindles.summary()
spindles_summary # simple statement for printing the output nicely

It found 793 spindles in total, and gives us quite a few parameters for these spindles (duration, amplitude, RMS, number of oscillations and so on! Looks promising.). Now lets calculate the mean of some of the values.

The variable `spindles_summary` is a so called `DataFrame`. You can access the columns simply by putting their names in the the squared brackets. On the columns dozens of functions are defined that you can easily run, such as `max(), min(), mean(), median(), sum(), quantile()` and many many more. 


In [ ]:
mean_duration = spindles_summary['Duration'].mean()
median_amplitude = spindles_summary['Amplitude'].median()
freq_quant_90 = spindles_summary['Frequency'].quantile(0.9)

# let's calculate the spindle density by dividing the total number of spindles by the minutes
minutes_in_data = raw.n_times / sampling_frequency / 60 # calculate minutes in data by number of samples divided by sampling_frequency and then by 60
spindle_density = len(spindles_summary) / minutes_in_data # length of spindles_summary=792 divided by minutes_in_data

In [ ]:
# lets print out these results

print('Mean spindle duration', mean_duration, 'seconds')
print('Median spindle amplitude', median_amplitude, 'uV')
print('90% quantile spindle frequency', freq_quant_90, 'Hz')
print('spindle density', spindle_density, 'spindles/seconds')

#### Now it's your turn! 

in the cell below try to fill in the code to calculate and print
- the mean abs power of all spindles
- the mean spindle frequency
- the 75 quantile of the absolute power

In [ ]:
mean_abs_power = ....
mean_spindle_freq = ...
quant_75 = ...


print(...)

## Run analysis on only certain sleep stages

Of course now the detection algorithm will find a lot of spindles that are not in the are of interest, e.g. REM or other artefacts that are no spindles at all. Therefore we want to limit our analysis to just run on certain segments of the data. For this we are loading the hypnogram of the specific file into memory.

- Make sure the hypnogram file (.TXT) is in the same folder as the EDF file and has the same name as the EDF file.

In [ ]:
# make sure the hypnogram of this participant is in the same folder as the EDF
hypnogram_file = data_dir + '/AA3_EX_AA3_EX_(1).edf.txt' # the hypnogram file
hypnogram = sleep_utils.read_hypno(hypnogram_file)

yasa.plot_hypnogram(hypnogram)
yasa.plot_spectrogram(data[0], sampling_frequency)

print('Now lets look at the hypnogram and the spectrogram of the data')

#### running YASA with specific sleep stages only

We can tell the YASA sleep spindle detection algorithm to only run on a certain segment of the data. We are basically doing the same analysis as above, we just pass some additional parameters to `yasa.spindles_detect()`. 

For this we first need to fit the hypnogram, which currently is in steps of 30 seconds such that each data point (i.e. 256 points per second) has one annotation. Then we provide the hypnogram to the `yasa.spindles_detect` function and tell it to only include certain stages.

Here the stages are defined as following : 
```
-2 = Unscored
-1 = Artefact / Movement
 0 = Wake
 1 = N1 sleep
 2 = N2 sleep
 3 = N3 sleep
 4 = REM sleep
```
IMPORTANT: Always make sure that these match your hypnogram! There is no standard in sleep research, and sometimes stages get annotated entirely differently. For example YASA only works with AASM, so there is no S4. In other annotation systems, `4` would annotate S4 and `5` would be REM, etc. So before analysing other data sets, make sure you have the data in the right format.

In [ ]:
stages_of_interest = [2, 3] # only run on NREM Stages 2 and 3 (SWS).
hypno = yasa.hypno_upsample_to_data(hypnogram, sf_hypno=1/30, data=data, sf_data=sampling_frequency)

spindles_NREM = yasa.spindles_detect(data, 
                                sf=sampling_frequency, 
                                hypno=hypno, 
                                include=stages_of_interest,
                                freq_sp=(12, 15),
                                verbose='INFO') 

print('Spindles found in S2 and S3')
spindles_NREM.summary()

As you can see, there were aroun ~180 spindles that were detected outside of NREM sleep. Now you could calculate the mean/median etc just as already done above, but this time using the DataFrame `spindles_NREM` as a basis.

#### Now it's your turn (2)! 
Now as an exercise, in the cell below try to calculate 
- the difference between the mean spindle frequency in S2 and S3!
- If you want, as a "bonus exercise", you can try to find a way to make another calculation:
   1. Calculate "slow" spindles by limiting freq_sp to (12,14) 
   2. Calculate "fast" spindles by limiting freq_sp to (14,16)
   3. See if there are more fast or slow spindles in the signal

In [ ]:
spindles_summary_s2 = ...
spindles_summary_s3 = ...

mean_spindle_freq_s2 = ...
mean_spindle_freq_s3 = ...

print('The mean S2 spindle freq is', mean_spindle_freq_s2, 'Hz')
print('The mean S3 spindle freq is', mean_spindle_freq_s3, 'Hz')

print('There is a absolute difference of ', abs(mean_spindle_freq_s2 - mean_spindle_freq_s3), 'Hz of spindles in S2 and S3')

### Summary

In this notebook you saw how to perform the spindle analysis on one participant. However you are of course interested in not only one participant, but in several ones. Additionally, you want the data in a format that can be read by other programs, e.g. XLS or CSV. Obviously you do not need to run each participant manually and extract values by copy&paste, but you can do so with a simple script.

The next notebook will provide the code to load and run data for several participants in one go.